In [1]:

import ee
import pandas as pd
import geemap
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:

# Authenticate and initialize Earth Engine
#ee.Authenticate()
ee.Initialize()

In [3]:

# Load CSV file with lat-long points
csv_path = r'data\africa\extracted_data\mean\agg_mean\TAHMO.csv'  # Update this if needed
df = pd.read_csv(csv_path)
df

,Unnamed: 0,sensor_id,network,station,sensor_name,latitude,longitude,elevation,depth_from,depth_to,...,2022-06-29,2022-06-30,2022-07-01,2022-07-02,2022-07-03,2022-07-04,2022-07-05,2022-07-06,2022-07-07,geometry
0,12,12,TAHMO,CoteDIvoireMeteoOffice,TEROS12,5.25891,-3.93513,99.0,0.10,0.10,...,0.141625,0.168250,0.159542,0.148250,0.137875,0.128750,0.140917,0.149917,0.158929,POINT (-3.93513 5.25891)
1,1,1,TAHMO,BiaSHTSDebiso,TEROS12,6.66462,-3.09725,228.0,0.10,0.10,...,0.297458,0.281875,0.263417,0.258292,0.251667,0.241375,0.231250,0.223333,0.220412,POINT (-3.09725 6.66462)
2,18,18,TAHMO,JuabosoSHS,TEROS12,6.34518,-2.82625,172.0,0.10,0.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-2.82625 6.34518)
3,13,13,TAHMO,"DaffiamaSHS,Daffiama",TEROS12,10.42496,-2.55448,330.0,0.10,0.10,...,0.152500,0.144417,0.136958,0.130542,0.124583,0.114708,0.103750,0.096500,0.092643,POINT (-2.55448 10.42496)
4,31,31,TAHMO,SacredHeartSHSNsoatre,TEROS12,7.41301,-2.47607,359.0,0.10,0.10,...,NaN,NaN,0.281833,0.288083,0.292833,0.275708,0.257500,0.245643,NaN,POINT (-2.47607 7.41301)
5,16,16,TAHMO,"HanSHS,Han",TEROS12,10.67041,-2.46477,320.0,0.10,0.10,...,0.262458,0.255208,0.235708,0.219208,0.203292,0.186167,0.173292,0.164833,0.159636,POINT (-2.46477 10.67041)
6,11,11,TAHMO,"CSIR-SARI,Tanina",TEROS12,9.88575,-2.46211,348.0,0.10,0.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-2.46211 9.88575)
7,33,33,TAHMO,"St.PetersRCJHS,St.JohnsRCJHS,Tuna",TEROS12,9.49418,-2.43478,319.0,0.10,0.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-2.43478 9.49418)
8,4,4,TAHMO,"BuiPowerAuthority,Bui",TEROS12,8.24704,-2.25272,171.0,0.10,0.10,...,0.133542,0.121792,0.115792,0.228250,0.201833,0.176042,0.152708,0.140208,0.134588,POINT (-2.25272 8.24704)
9,36,36,TAHMO,"WenchiMethodistSHS,Wenchi",TEROS12,7.75559,-2.10156,322.0,0.10,0.10,...,0.282333,0.309375,0.303583,0.313958,0.305708,0.296917,0.314958,0.304000,NaN,POINT (-2.10156 7.75559)


In [4]:

# Ensure column names
lat_col, lon_col = 'latitude', 'longitude'  # Modify if your columns are named differently

# Extract bounding box
min_lat = df[lat_col].min()
max_lat = df[lat_col].max()
min_lon = df[lon_col].min()
max_lon = df[lon_col].max()

# Create EE geometry bounding box
region_bbox = ee.Geometry.BBox(min_lon, min_lat, max_lon, max_lat)
print(f"✅ Bounding box created: ({min_lon}, {min_lat}) to ({max_lon}, {max_lat})")

✅ Bounding box created: (-3.93513, -2.88283) to (40.02984, 11.06886)


In [5]:
features = []
for _, row in df.iterrows():
    point = ee.Geometry.Point(row[lon_col], row[lat_col])
    feature = ee.Feature(point, {
        'Latitude': row[lat_col],
        'Longitude': row[lon_col]
    })
    features.append(feature)
point_fc = ee.FeatureCollection(features)


In [6]:
# Sentinel-2 NDVI calculation function
def compute_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

# Define cloud masking function using QA60 band (clouds and cirrus)
def mask_s2_clouds(image):
    qa = image.select('QA60')
    # Bits 10 and 11 are clouds and cirrus
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(
           qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).copyProperties(image, ['system:time_start'])



In [7]:

# Date range
start_date = '2016-06-01'
end_date = '2025-06-14'


In [8]:
# Load Sentinel-2 surface reflectance, apply filters
s2_col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(region_bbox) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(mask_s2_clouds) \
    .map(compute_ndvi) \
    .select('NDVI')

In [ ]:
# # Print metadata: number of images
# count = s2_col.size().getInfo()
# print(f"✅ Total images in filtered collection: {count}")

In [9]:

# Function to extract NDVI time series for a point
def extract_ndvi_series(lat, lon):
    point = ee.Geometry.Point(float(lon), float(lat))
    def extract_value(image):
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
        value = image.reduceRegion(ee.Reducer.mean(), point, scale=10).get('NDVI')
        return ee.Feature(None, {'date': date, 'NDVI': value})
    features = s2_col.map(extract_value).filter(ee.Filter.notNull(['NDVI']))
    return features.aggregate_array('date').getInfo(), features.aggregate_array('NDVI').getInfo()


In [10]:
# Get and display metadata of the first image
first_img = s2_col.first()
first_info = first_img.getInfo()
print("\n📄 Metadata of first image in collection:")
print(f"ID: {first_info['id']}")
print(f"Date: {ee.Date(first_img.get('system:time_start')).format('YYYY-MM-dd').getInfo()}")

# Visualize NDVI of first image
Map = geemap.Map(center=[10, 0], zoom=4)  # Adjust center for your region
ndvi_vis = {'min': 0, 'max': 1, 'palette': ['blue', 'white', 'green']}
Map.addLayer(first_img.select('NDVI'), ndvi_vis, 'First NDVI Image')
Map.addLayer(region_bbox,{},'BBOX')
Map.addLayer(point_fc, {'color': 'red'}, 'Stations')
Map.add_colorbar(vis_params=ndvi_vis, label='NDVI', layer_name='First NDVI Image')
Map


📄 Metadata of first image in collection:
ID: COPERNICUS/S2_SR_HARMONIZED/20171209T095359_20171209T100602_T32PLR
Date: 2017-12-09


Map(center=[10, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

In [ ]:

# Prepare output dataframe
results = []

for idx, row in tqdm(df.iterrows(), total=len(df)):
    lat, lon, network,station = row[lat_col], row[lon_col], row['network'], row['station']
    try:
        dates, values = extract_ndvi_series(lat, lon)
        row_data = {
            'network':network,
            'station':station,
            'Latitude': lat,
            'Longitude': lon
        }
        row_data.update({date: value for date, value in zip(dates, values)})
        results.append(row_data)
    except Exception as e:
        print(f"Error at index {idx}: {e}")

# Convert result to DataFrame
ndvi_df = pd.DataFrame(results)


  3%|▎         | 1/33 [05:01<2:40:39, 301.25s/it]

Error at index 0: Computation timed out.
